In [1]:
from pycocotools.coco import COCO
import requests


In [2]:
import numpy as np

In [3]:
coco = COCO('./coco_images/annotations/instances_train2017.json')

loading annotations into memory...
Done (t=13.16s)
creating index...
index created!


In [4]:
cats = coco.loadCats(coco.getCatIds())

In [5]:
nms = [cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush



In [6]:
# catIds = coco.getCatIds(catNms=['person','dog','bicycle', 'car', 'skateboard', 'cat', 'horse', 'bus'])

In [7]:
catIds = coco.getCatIds(catNms=['person','dog'])
catIds

[1, 18]

In [8]:
catlist = coco.loadCats(catIds)
catlist

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'}]

In [9]:
imgIds = coco.getImgIds(catIds=catIds)

In [10]:
images = coco.loadImgs(imgIds)

In [11]:
images[0]

{'license': 3,
 'file_name': '000000524291.jpg',
 'coco_url': 'http://images.cocodataset.org/train2017/000000524291.jpg',
 'height': 426,
 'width': 640,
 'date_captured': '2013-11-18 09:59:07',
 'flickr_url': 'http://farm2.staticflickr.com/1045/934293170_d1b2cc58ff_z.jpg',
 'id': 524291}

In [12]:
len(images)

2049

In [13]:
import matplotlib

In [15]:
color_list = [matplotlib.colors.to_hex(color)  for color in list(matplotlib.colors.TABLEAU_COLORS)]

In [16]:
# color_list = list(matplotlib.colors.cnames.values())

In [17]:
color_list[:2]

['#1f77b4', '#ff7f0e']

In [18]:
from sklearn.model_selection import train_test_split

In [20]:
train_image, test_image = train_test_split(images, test_size = 0.2)

In [21]:
print(len(train_image), len(test_image))

1639 410


In [22]:
import json
from tqdm import tqdm
import os

In [46]:
cores = os.cpu_count()

In [23]:
# tt = './Deep-POC-2019/coco_images/downloaded_images/annotations_download/'

In [24]:
# os.path.join(tt, 'train')

In [25]:
def json_generator(dir_path,  catlist, images, train=True):
    if train:
        save_path = os.path.join(dir_path,'train')
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        save_name = save_path + '/' + 'annotation_train.json'
    else:
        save_path = os.path.join(dir_path,'test')
        save_name = save_path + '/' + 'annotation_test.json'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
#     save_name = save_path + '/' + 'annotation_'
    with open(save_name,'w') as json_file:
        ann_data = {}


        categories = []
        for cat_idx, cat in enumerate(catlist):
    #     print(cat_idx)
            cat['color'] = color_list[cat_idx]
            categories.append(cat)
            print(cat)

        ann_data['categories'] =categories 

        ann_data['images'] = images

        annotations = []
        for im in tqdm(images):
            for cat_idx,  cat in enumerate(catIds):
                annIds = coco.getAnnIds(imgIds=im['id'], catIds=cat, iscrowd=None)
                anns = coco.loadAnns(annIds)
                for i in range(len(anns)):
                    ann_mini = anns[i]
                    ann_mini['color'] = color_list[cat_idx]

                    annotations.append(ann_mini)
        ann_data['annotations'] = annotations
        json.dump(ann_data, json_file)

        json_file.close()

In [26]:
json_generator('./coco_images/downloaded_images/annotations_download',
              catlist=catlist, images=train_image, train=True)

100%|██████████| 1639/1639 [00:00<00:00, 47251.40it/s]

{'supercategory': 'person', 'id': 1, 'name': 'person', 'color': '#1f77b4'}
{'supercategory': 'animal', 'id': 18, 'name': 'dog', 'color': '#ff7f0e'}


In [27]:
json_generator('./coco_images/downloaded_images/annotations_download',
              catlist=catlist, images=test_image, train=False)

100%|██████████| 410/410 [00:00<00:00, 42752.20it/s]

{'supercategory': 'person', 'id': 1, 'name': 'person', 'color': '#1f77b4'}
{'supercategory': 'animal', 'id': 18, 'name': 'dog', 'color': '#ff7f0e'}


In [30]:
# with open('./coco_images/downloaded_images/annotations_download_train.json','w') as json_file:
#     ann_data = {}
    
    
#     categories = []
#     for cat_idx, cat in enumerate(catlist):
# #     print(cat_idx)
#         cat['color'] = color_list[cat_idx]
#         categories.append(cat)
#         print(cat)
        
#     ann_data['categories'] =categories 
    
#     ann_data['images'] = images
        
#     annotations = []
#     for im in tqdm(images):
#         for cat_idx,  cat in enumerate(catIds):
#             annIds = coco.getAnnIds(imgIds=im['id'], catIds=cat, iscrowd=None)
#             anns = coco.loadAnns(annIds)
#             for i in range(len(anns)):
#                 ann_mini = anns[i]
#                 ann_mini['color'] = color_list[cat_idx]
                
#                 annotations.append(ann_mini)
#     ann_data['annotations'] = annotations
#     json.dump(ann_data, json_file)
    
#     json_file.close()

In [31]:
# coco_new = COCO('./coco_images/downloaded_images/annotations_download_test.json')

In [32]:
# coco_new_cats = coco_new.loadCats(coco_new.getCatIds())

In [33]:
# coco_image_ids  = coco_new.getImgIds()

In [34]:
# len(coco_image_ids)

In [35]:
# coco_new_images = coco_new.loadImgs(coco_image_ids)

In [36]:
# len(coco_new_images)

In [38]:
# os.path.join('./coco_images/downloaded_images' + '/images/','train')

In [39]:
# train, test = train_test_split(coco_new_images, test_size = 0.2)

In [40]:
from multiprocessing import Pool
from functools import partial

In [41]:
def image_download(image_dict, dir_path='./train_images', train=True):
    if train:
        save_path = os.path.join(dir_path + '/images/','train/')
        if not os.path.exists(save_path):
            os.makedirs(save_path)
#         save_name = save_path + '/' + 'annotation_train.json'
    else:
        save_path = os.path.join(dir_path + '/images/','test/')
#         save_name = save_path + '/' + 'annotation_test.json'
        if not os.path.exists(save_path):
            os.makedirs(save_path)
    
    for im in tqdm(image_dict):    
        print("im : ", im)
    
        img_data = requests.get(im['coco_url']).content
        with open(save_path + im['file_name'], 'wb') as handler:
            handler.write(img_data)
#     print('done')


# def json_generator(dir_path,  catlist, images, train=True):
#     if train:
#         save_path = os.path.join(dir_path,'train')
#         if not os.path.exists(save_path):
#             os.makedirs(save_path)
#         save_name = save_path + '/' + 'annotation_train.json'
#     else:
#         save_path = os.path.join(dir_path,'test')
#         save_name = save_path + '/' + 'annotation_test.json'
#         if not os.path.exists(save_path):
#             os.makedirs(save_path)

In [42]:
# for im in images:
#     print("im: ", im)
#     img_data = requests.get(im['coco_url']).content
#     with open('./Deep-POC-2019/coco_images/downloaded_images/' + im['file_name'], 'wb') as handler:
#         handler.write(img_data)

In [43]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [44]:
with Pool(cores) as p:
    chunks_images = list(chunks(train_image, int(len(train_image)/ cores)))
    p.map(partial(image_download, dir_path = './coco_images/downloaded_images', train=True),  chunks_images)

  0%|          | 0/204 [00:00<?, ?it/s]

im :  {'license': 3, 'file_name': '000000315555.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000315555.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-18 11:11:18', 'flickr_url': 'http://farm8.staticflickr.com/7098/7386751942_e6babf9acd_z.jpg', 'id': 315555}


  0%|          | 0/204 [00:00<?, ?it/s]

im :  {'license': 1, 'file_name': '000000338120.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000338120.jpg', 'height': 640, 'width': 426, 'date_captured': '2013-11-17 22:14:50', 'flickr_url': 'http://farm8.staticflickr.com/7193/7059651849_393b9af791_z.jpg', 'id': 338120}


  0%|          | 0/204 [00:00<?, ?it/s]

...

 86%|████████▌ | 6/7 [00:07<00:01,  1.24s/it]

im :  {'license': 1, 'file_name': '000000154062.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000154062.jpg', 'height': 375, 'width': 500, 'date_captured': '2013-11-16 19:27:48', 'flickr_url': 'http://farm1.staticflickr.com/32/51988573_69442b9868_z.jpg', 'id': 154062}


100%|█████████▉| 203/204 [04:04<00:01,  1.14s/it]

im :  {'license': 3, 'file_name': '000000006710.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000006710.jpg', 'height': 480, 'width': 640, 'date_captured': '2013-11-17 22:15:19', 'flickr_url': 'http://farm1.staticflickr.com/86/385884682_d322eec86f_z.jpg', 'id': 6710}


100%|██████████| 204/204 [04:05<00:00,  1.20s/it]


In [45]:
with Pool(cores) as p:
    chunks_images = list(chunks(test_image, int(len(test_image)/ cores)))
    p.map(partial(image_download, dir_path = './coco_images/downloaded_images', train=False),  chunks_images)

  0%|          | 0/51 [00:00<?, ?it/s]

im :  {'license': 6, 'file_name': '000000413245.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000413245.jpg', 'height': 457, 'width': 640, 'date_captured': '2013-11-18 12:23:45', 'flickr_url': 'http://farm7.staticflickr.com/6080/6109873174_f13f113eb7_z.jpg', 'id': 413245}


  0%|          | 0/51 [00:00<?, ?it/s]

im :  {'license': 1, 'file_name': '000000001706.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000001706.jpg', 'height': 518, 'width': 640, 'date_captured': '2013-11-18 14:25:02', 'flickr_url': 'http://farm5.staticflickr.com/4111/5041936760_436558a44d_z.jpg', 'id': 1706}


  0%|          | 0/51 [00:00<?, ?it/s]

...

 98%|█████████▊| 50/51 [01:01<00:01,  1.18s/it]

im :  {'license': 3, 'file_name': '000000355778.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000355778.jpg', 'height': 480, 'width': 640, 'date_captured': '2013-11-14 17:40:30', 'flickr_url': 'http://farm3.staticflickr.com/2149/2469639035_dbb43453bf_z.jpg', 'id': 355778}


 98%|█████████▊| 50/51 [01:02<00:01,  1.18s/it]

im :  {'license': 3, 'file_name': '000000288157.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000288157.jpg', 'height': 363, 'width': 640, 'date_captured': '2013-11-18 09:40:13', 'flickr_url': 'http://farm4.staticflickr.com/3061/2847226576_f3117b14d4_z.jpg', 'id': 288157}


100%|██████████| 51/51 [01:03<00:00,  1.24s/it]
